In [1]:
import os
import google.generativeai as genai
from dotenv import load_dotenv

load_dotenv()

genai.configure(api_key=os.environ["GEMINI_API_KEY"])

In [2]:
# Create the model
generation_config = {
    "temperature": 1,
    "top_p": 0.95,
    "top_k": 40,
    "max_output_tokens": 8192,
    "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
    model_name="gemini-2.0-flash-exp",
    generation_config=generation_config,
)

In [3]:
chat_session = model.start_chat(history=[])

In [44]:
prompt = """
### 指示 ###
あなたは製品の事故原因を分析しています。
事故の通知内容から、被害の様子を単純に説明してください。
回答形式はJSON形式です。

### 回答形式 ###
{
	"被害":"回答"
}

### 注意事項 ###
- 指示された回答のみ出力してください。
- 被害の様子を最もよく説明する単語のみで回答してください。
- 被害は複数回答してもかまいません。その場合、配列で記述してください。
- 解説は不要です。

### 例 ###
事故の通知内容:
カセットこんろの五徳付プレートを洗っていたところ、指が切れ、第一関節裏側を2針縫った。

回答:
{
	"被害":"裂傷"
}

### 例の解説 ###
「指が切れ」という記述から、「裂傷」を負ったと判断できます。

### 事故の通知内容 ###
"""

In [5]:
import pandas as pd
import json

In [46]:
data = pd.read_csv("data/output/gemini/prompt_data/ミキサー.csv")

In [41]:
input_prompt = prompt + data["事故通知内容"].values[0]
print(input_prompt)


### 指示 ###
あなたは製品の事故原因を分析しています。
事故の通知内容から、被害の様子を単純に説明してください。
回答形式はJSON形式です。

### 回答形式 ###
{
	"被害":"回答"
}

### 注意事項 ###
- 指示された回答のみ出力してください。
- 被害の様子を最もよく説明する単語のみで回答してください。
- 被害は複数回答してもかまいません。その場合、配列で記述してください。
- 解説は不要です。

### 例 ###
事故の通知内容:
カセットこんろの五徳付プレートを洗っていたところ、指が切れ、第一関節裏側を2針縫った。

回答:
{
	"被害":"裂傷"
}

### 例の解説 ###
「指が切れ」という記述から、「裂傷」を負ったと判断できます。

### 事故の通知内容 ###
ハンドミキサーを使用して卵と砂糖を泡立てていたところ、泡立て金具が1本破損して指にあたった。


In [12]:
response = chat_session.send_message(input_prompt)
print(response.text)

{
	"被害":["圧痕","破断"]
}



In [14]:
res = json.loads(response.text)
for text in res["被害"]:
    print(text)

圧痕
破断


In [15]:
output_text = ""
for text in res["被害"]:
    output_text += text + " "
data["事故通知内容"].values[0] = output_text

In [16]:
print(data["事故通知内容"].values[0])

圧痕 破断 


In [17]:
from tqdm.notebook import tqdm
import time

In [47]:
for i in tqdm(range(0, len(data))):
    input_prompt = prompt + data["事故通知内容"].values[i]
    response = chat_session.send_message(input_prompt)
    temp = response.text
    if "```json" in temp:
        temp = temp.replace("```json", "")
        temp = temp.replace("```", "")
    try:
        res = json.loads(temp)
    except Exception as e:
        print("Error", e, temp)
        continue
    output_text = ""
    for text in res["被害"]:
        output_text += text + " "
    data["事故通知内容"].values[i] = output_text
    time.sleep(5)

  0%|          | 0/19 [00:00<?, ?it/s]

In [48]:
prompt = """
### 指示 ###
あなたは製品の事故原因を分析しています。
事故の内容から、事故の原因を単純に説明してください。
回答形式はJSON形式です。

### 回答形式 ###
{
	"原因":"回答"
}

### 注意事項 ###
- 指示された回答のみ出力してください。
- 事故の原因を最もよく説明する単語のみで回答してください。
- 原因は複数回答してもかまいません。その場合、配列で記述してください。
- 解説は不要です。

### 例 ###
事故の内容:
五徳付プレートの外側の縁が処理されておらず、多少ざらついていることから、素手でプレートを洗っていた際に、外側の縁に指を強く擦りつけたため、負傷したものと推定される。なお、取扱説明書には「手、指の保護のため、必ずゴム手袋などを使用してください。」との注意が記載されている。

回答:
{
	"原因":["縁","接触"]
}

### 例の解説 ###
「縁に指を強く擦りつけた」という記述から、「縁」、「接触」が原因であると判断できます。

### 事故の内容 ###
"""

In [49]:
for i in tqdm(range(0, len(data))):
    input_prompt = prompt + data["事故原因"].values[i]
    response = chat_session.send_message(input_prompt)
    temp = response.text
    if "```json" in temp:
        temp = temp.replace("```json", "")
        temp = temp.replace("```", "")
    try:
        res = json.loads(temp)
    except Exception as e:
        print("Error", e, temp)
        continue
    output_text = ""
    for text in res["原因"]:
        output_text += text + " "
    data["事故原因"].values[i] = output_text
    time.sleep(5)

  0%|          | 0/19 [00:00<?, ?it/s]

In [50]:
print(data.head())

      No  事故通知内容          事故原因
0    576  破損 接触   圧搾機 圧痕 金属疲労 
1   2797    切 創         誤 操 作 
2   4253    切 創        電源 誤操作 
3  12164  発煙 異臭         過 負 荷 
4  22370    混 入       金型 金属疲労 


In [51]:
data.to_csv("data/output/gemini/responses/ミキサー_res.csv", index=False)